In [ ]:
!wget https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
!pip install fasttext
!pip install -U pip transformers
!pip install sentencepiece

--2023-08-30 13:13:44--  https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.251.106, 52.84.251.15, 52.84.251.27, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.251.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1176355829 (1.1G) [application/octet-stream]
Saving to: ‘lid218e.bin.4’

lid218e.bin.4       100%[===================>]   1.09G   153MB/s    in 9.0s    

2023-08-30 13:13:53 (124 MB/s) - ‘lid218e.bin.4’ saved [1176355829/1176355829]



In [ ]:
# Language Identification Pipeline
import fasttext

pretrained_lang_model = "/content/lid218e.bin" # Path of model file
lang_detection_model = fasttext.load_model(pretrained_lang_model)

text = "صباح الخير، الجو جميل اليوم والسماء صافية."
predictions = lang_detection_model.predict(text, k=1)
print(predictions) #(('__label__arb_Arab'), array([0.99960977]))

(('__label__arb_Arab',), array([0.99960977]))


In [ ]:
# Load Translation Models
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

checkpoint = 'facebook/nllb-200-distilled-600M'
# checkpoint = 'facebook/nllb-200–1.3B'
# checkpoint = 'facebook/nllb-200-3.3B'
# checkpoint = 'facebook/nllb-200-distilled-1.3B'

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, cache_dir='/models')
tokenizer = AutoTokenizer.from_pretrained(checkpoint, cache_dir='/models')

model.to('cuda')
# To reduce GPU RAM Usage
model.half()

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm

In [ ]:
# Configure pipeline
target_lang = 'eng_Latn'
source_lang = 'arb_Arab'

# Check if a GPU is available
if torch.cuda.is_available():
    device = 0  # Use GPU 0 (you can change this if you have multiple GPUs)
else:
    device = -1  # Use CPU if GPU is not available
print('device =', device)

# translator_pipeline = pipeline(
#     'translation',
#     model=model,
#     tokenizer=tokenizer,
#     device=device,
#     src_lang=source_lang,
#     tgt_lang=target_lang,
#     max_length = 512,
#     batch_size = 16
#   )

device = 0


In [ ]:
# Get comments dataframe
import pandas as pd

csv_url = '''
https://raw.githubusercontent.com/microSight/AIFeatures/main/data/comments_8iAFFE56z_8.csv?token=GHSAT0AAAAAACG6EHWSZGHSRFTJYGGUSK2CZHPIALA
'''
df = pd.read_csv(csv_url)
df

,video_id,comment,comment_publish_date,like_count,author_name
0,8iAFFE56z_8,Bu adamı dinledikce tv lerdeki boş teneke insa...,2023-07-17T10:51:14Z,0,Taha Bolat
1,8iAFFE56z_8,Hakan hocanın kendi kanalı ders sitesi VS var ...,2023-07-14T21:28:21Z,0,Zeynep Akbulut
2,8iAFFE56z_8,Sana bir şey olmasın işin özü,2023-07-08T15:25:27Z,0,Mustafa Alparslan İMAMOĞLU
3,8iAFFE56z_8,Hocam valla keşke her ekonomi hocası sizin gib...,2023-06-30T13:13:08Z,0,Senior Software Engineer
4,8iAFFE56z_8,Meslegi ne doktor ne hakkinda konusuyo ekonomi...,2023-06-21T00:04:59Z,0,Lakelos
...,...,...,...,...,...
300,8iAFFE56z_8,Sonunda,2023-03-31T13:03:27Z,1,Reginaiere
301,8iAFFE56z_8,Yürü bee en sevdiğim….,2023-03-31T13:03:10Z,3,Murad
302,8iAFFE56z_8,Peki ne yapalim o zaman...,2023-03-31T13:02:54Z,2,Nurlan Samedov
303,8iAFFE56z_8,selamlar özledik be hakan hocam,2023-03-31T13:02:25Z,2,Cenk Eren Erdi


In [ ]:
# Lang identification and translation to english pipeline function
import gc

def pipee(df):
  # Get lang_label using fasttext model
  # ---------------------------------------------------------------
  # Fasttext models requires no '\n' char
  df['translated_comment'] = df['comment'].astype('str').apply(lambda x: x.replace('\n',' '))

  # Get lang detection predictions
  lang_id_output = lang_detection_model.predict(df['translated_comment'].tolist())
  df['lang_label'] = list(map(lambda x: x[0].replace('__label__', '') ,lang_id_output[0]))

  # Get translations using huggingface pipeline
  # --------------------------------------------------------------
  for lang_label in df['lang_label'].unique().tolist():
    input_comments = df.loc[ df['lang_label'] == lang_label,  'translated_comment'].tolist()

    translator_pipeline = pipeline(
        'translation',
        model=model,
        tokenizer=tokenizer,
        device=device,
        src_lang=lang_label,
        tgt_lang='eng_Latn',
        max_length = 512,
        batch_size = 16
      )

    output = translator_pipeline(input_comments)
    df.loc[ df['lang_label'] == lang_label,  'translated_comment'] = list(map(lambda x: x['translation_text'], output))

    del translator_pipeline
    torch.cuda.empty_cache()
    gc.collect()


  return df

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(checkpoint)
max_length = config.max_position_embeddings
max_length

1024

In [ ]:
# Apply pipeline to dataframe and add 'translated_comment' and 'lang_label' columns to dataframe
df = pipee(df)
df

,video_id,comment,comment_publish_date,like_count,author_name,translated_comment,lang_label
0,8iAFFE56z_8,Bu adamı dinledikce tv lerdeki boş teneke insa...,2023-07-17T10:51:14Z,0,Taha Bolat,"Listening to this guy, I have no respect for t...",tur_Latn
1,8iAFFE56z_8,Hakan hocanın kendi kanalı ders sitesi VS var ...,2023-07-14T21:28:21Z,0,Zeynep Akbulut,Do you guys have your own channel of lessons?,tur_Latn
2,8iAFFE56z_8,Sana bir şey olmasın işin özü,2023-07-08T15:25:27Z,0,Mustafa Alparslan İMAMOĞLU,♪ Let nothing happen to you ♪,tur_Latn
3,8iAFFE56z_8,Hocam valla keşke her ekonomi hocası sizin gib...,2023-06-30T13:13:08Z,0,Senior Software Engineer,"Master, I wish every economics teacher was as ...",tur_Latn
4,8iAFFE56z_8,Meslegi ne doktor ne hakkinda konusuyo ekonomi...,2023-06-21T00:04:59Z,0,Lakelos,It's not about the profession or the doctor or...,tur_Latn
...,...,...,...,...,...,...,...
300,8iAFFE56z_8,Sonunda,2023-03-31T13:03:27Z,1,Reginaiere,Finally,tur_Latn
301,8iAFFE56z_8,Yürü bee en sevdiğim….,2023-03-31T13:03:10Z,3,Murad,My favorite walk is...,tur_Latn
302,8iAFFE56z_8,Peki ne yapalim o zaman...,2023-03-31T13:02:54Z,2,Nurlan Samedov,So what do I do?,tur_Latn
303,8iAFFE56z_8,selamlar özledik be hakan hocam,2023-03-31T13:02:25Z,2,Cenk Eren Erdi,"We missed you, my dear teacher.",tur_Latn


In [ ]:
# Check original and translated comments
for idx, row in df.sort_values('like_count', ascending=False)[['comment', 'translated_comment']].iterrows():
  print(row['comment'])
  print(row['translated_comment'])
  print('')

Hakan hocam ekonomi denilen şu karmaşık sistemi sizin kadar eğlenceli bir şekilde anlatan başka birisini tanımadık. Teşekkürler.
Professor Hakan, we've never met anyone else who has described this complex system called economics in a more entertaining way than you.

özlemiştik bu programı.
We missed this show.

tüm zamanların en eğitici ve eğlenceli ekonomi programını siz yapıyorsunuz hocam.
You're the one who's doing the most educational and entertaining economics program of all time, Professor.

İstediğim program gelmiş, 2000'lerde Kral TV de istediği parça çalınmış insanlar gibi sevindim. Elinize sağlık hocam.
I got the show I wanted, and I was happy like the people who stole the show from the 2000s on King TV.

Hakan hocam , bize ekonomi dinlemeyi sevdirdin. Iyi ki varsın, Kıbrıstan selamlar.
Master Hakan, you made us like to listen to economics.

2-3 yıllık bir bankacı olarak Hakan beyi keyifle dinliyorum, böyle bir içeriği bize ulaştırdığı için Flu tv'ye de ayrıca teşekkür ederim